In [ ]:
%load_ext autoreload
%autoreload 2

from qiskit_metal import MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6

from qiskit_metal.renderers.renderer_gmsh.gmsh_renderer import QGmshRenderer

from qiskit_metal.renderers.renderer_elmer.elmer_renderer import QElmerRenderer
from qiskit_metal.renderers.renderer_elmer.elmer_renderer import load_capacitance_matrix_from_file

In [2]:

design = designs.MultiPlanar({}, True)


gui = MetalGUI(design)

In [3]:

conn_pads = dict(
    connection_pads = dict(
        readout = dict(loc_W=0, loc_H=-1),
        coupler1 = dict(loc_W=-1, loc_H=1),
        coupler2 = dict(loc_W=1, loc_H=1)
    )
)


q1 = TransmonPocket6(design, "Q1", options=dict(**conn_pads))


gui.rebuild()
gui.autoscale()

In [4]:

gmsh_renderer = QGmshRenderer(design)


gmsh_renderer.render_design(open_pins=[("Q1", "coupler1"), ("Q1", "coupler2"), ("Q1", "readout")],
                            mesh_geoms=False)

In [5]:

gmsh_renderer.launch_gui()

In [6]:

gmsh_renderer.add_mesh(dim=3, intelli_mesh=False)
gmsh_renderer.launch_gui()

In [ ]:

gmsh_renderer.options.mesh.min_size = '5um'
gmsh_renderer.options.mesh.max_size = '20um'

gmsh_renderer.render_design(open_pins=[("Q1", "coupler1"), ("Q1", "coupler2"), ("Q1", "readout")],
                            mesh_geoms=False)

gmsh_renderer.add_mesh(intelli_mesh=False)
gmsh_renderer.launch_gui()

In [ ]:

gmsh_renderer.options.mesh.min_size = '5um'
gmsh_renderer.options.mesh.max_size = '50um'

gmsh_renderer.render_design(open_pins=[("Q1", "coupler1"), ("Q1", "coupler2"), ("Q1", "readout")],
                            mesh_geoms=True)
gmsh_renderer.launch_gui()

In [7]:

gmsh_renderer.export_mesh("test.msh")

In [8]:

gmsh_renderer.close()

True

In [9]:

elmer_renderer = QElmerRenderer(design)

elmer_renderer.gmsh.options.mesh.min_size = "5um"
elmer_renderer.gmsh.options.mesh.max_size = "50um"

In [10]:

elmer_renderer.render_design(open_pins=[("Q1", "coupler1"), ("Q1", "coupler2"), ("Q1", "readout")],
                             skip_junctions=True)

In [11]:

elmer_renderer.launch_gmsh_gui()

In [ ]:
elmer_renderer.export_mesh()

In [13]:

elmer_renderer.add_solution_setup('capacitance')

In [14]:

elmer_renderer.run('capacitance')

08:04PM 44s INFO [run]: Running ElmerGrid on input mesh from Gmsh...
08:04PM 45s INFO [run]: Running ElmerSolver for solver type: 'capacitance'


In [15]:
elmer_renderer.capacitance_matrix

,Q1_readout_connector_pad,Q1_coupler1_connector_pad,Q1_coupler2_connector_pad,Q1_pad_top,Q1_pad_bot,ground_plane
Q1_readout_connector_pad,66.744094,-0.217009,-0.216818,-1.821051,-18.588840,-45.900376
Q1_coupler1_connector_pad,-0.217009,61.086923,-0.300955,-16.198516,-1.653818,-42.716625
Q1_coupler2_connector_pad,-0.216818,-0.300955,61.505852,-16.227055,-1.646792,-43.114232
Q1_pad_top,-1.821051,-16.198516,-16.227055,110.765851,-37.257608,-39.261620
Q1_pad_bot,-18.588840,-1.653818,-1.646792,-37.257608,104.211499,-45.064442
ground_plane,-45.900376,-42.716625,-43.114232,-39.261620,-45.064442,300.000000


In [16]:

elmer_renderer.display_post_processing_data()

08:05PM 25s INFO [model]: Added new model 'post_processing' and set as current.


In [17]:

elmer_renderer.save_capacitance_matrix("cap_matrix_official.txt")

elmer_renderer.close()

In [18]:

import numpy as np
from qiskit_metal.analyses.quantization.lom_core_analysis import CompositeSystem, Cell, Subsystem 
from scipy.constants import speed_of_light as c_light
import matplotlib.pyplot as plt
%matplotlib inline

08:07PM 43s INFO [__init__]: TransmonBuilder with system_type TRANSMON registered to QuantumSystemRegistry
08:07PM 43s INFO [__init__]: FluxoniumBuilder with system_type FLUXONIUM registered to QuantumSystemRegistry
08:07PM 43s INFO [__init__]: TLResonatorBuilder with system_type TL_RESONATOR registered to QuantumSystemRegistry
08:07PM 43s INFO [__init__]: LumpedResonatorBuilder with system_type LUMPED_RESONATOR registered to QuantumSystemRegistry


In [19]:

cap_matrix = load_capacitance_matrix_from_file("cap_matrix_officiaL.txt")

In [20]:

opt1 = dict(
    node_rename = {'Q1_coupler1_connector_pad': 'coupler1', 'Q1_coupler2_connector_pad': 'coupler2', 'Q1_readout_connector_pad': 'readout'}, 
    cap_mat = cap_matrix,
    ind_dict = {('Q1_pad_top', 'Q1_pad_bot'): 12},  
    jj_dict = {('Q1_pad_top', 'Q1_pad_bot'): 'j1'},
    cj_dict = {('Q1_pad_top', 'Q1_pad_bot'): 2}, 
)
cell_1 = Cell(opt1)

In [21]:

transmon = Subsystem(name='transmon1_sys', sys_type='TRANSMON', nodes=['j1'])

q_opts = dict(
    f_res = 7.4, 
    Z0 = 50,
    vp = 0.404314 * c_light
)
coup1 = Subsystem(name='coup1_sys', sys_type='TL_RESONATOR', nodes=['coupler1'], q_opts=q_opts)

q_opts = dict(
    f_res = 7.2, 
    Z0 = 50,
    vp = 0.404314 * c_light 
)
coup2 = Subsystem(name='coup2_sys', sys_type='TL_RESONATOR', nodes=['coupler2'], q_opts=q_opts)

q_opts = dict(
    f_res = 7,
    Z0 = 50, 
    vp = 0.404314 * c_light 
)
readout = Subsystem(name='readout_sys', sys_type='TL_RESONATOR', nodes=['readout'], q_opts=q_opts)

In [22]:

composite_sys = CompositeSystem(
    subsystems=[transmon, coup1, coup2, readout], 
    cells=[cell_1], 
    grd_node='ground_plane',
    nodes_force_keep=['coupler1', 'coupler2', 'readout']
)

In [23]:

cg = composite_sys.circuitGraph()
print(cg)

node_jj_basis:
-------------

['j1', 'Q1_pad_bot', 'coupler1', 'coupler2', 'readout']

nodes_keep:
-------------

['j1', 'coupler1', 'coupler2', 'readout']


L_inv_k (reduced inverse inductance matrix):
-------------

                j1  coupler1  coupler2  readout
j1        0.083333       0.0       0.0      0.0
coupler1  0.000000       0.0       0.0      0.0
coupler2  0.000000       0.0       0.0      0.0
readout   0.000000       0.0       0.0      0.0

C_k (reduced capacitance matrix):
-------------

                 j1   coupler1   coupler2    readout
j1        74.296680  -6.855830  -6.873110   8.860085
coupler1  -6.855830  58.817943  -2.572670  -2.811047
coupler2  -6.873110  -2.572670  59.231400  -2.813983
readout    8.860085  -2.811047  -2.813983  63.778429




In [24]:

hilbertspace = composite_sys.create_hilbertspace()
print(hilbertspace)

 c:\Users\itsme\anaconda3\envs\qmetal\lib\site-packages\qiskit_metal\analyses\quantization\lom_core_analysis.py: 170
 c:\Users\itsme\anaconda3\envs\qmetal\lib\site-packages\qiskit_metal\analyses\quantization\lom_core_analysis.py: 170
 c:\Users\itsme\anaconda3\envs\qmetal\lib\site-packages\qiskit_metal\analyses\quantization\lom_core_analysis.py: 170


HilbertSpace:  subsystems
-------------------------

Transmon------------| [Transmon_1]
                    | EJ: 13621.792733898432
                    | EC: 270.6303337815318
                    | ng: 0.001
                    | ncut: 22
                    | truncated_dim: 10
                    |
                    | dim: 45


Oscillator----------| [Oscillator_1]
                    | E_osc: 7400.0
                    | l_osc: None
                    | truncated_dim: 3
                    |
                    | dim: 3


Oscillator----------| [Oscillator_2]
                    | E_osc: 7200.0
                    | l_osc: None
                    | truncated_dim: 3
                    |
                    | dim: 3


Oscillator----------| [Oscillator_3]
                    | E_osc: 7000
                    | l_osc: None
                    | truncated_dim: 3
                    |
                    | dim: 3




In [25]:

hilbertspace = composite_sys.add_interaction()

hilbertspace.hamiltonian()

 c:\Users\itsme\anaconda3\envs\qmetal\lib\site-packages\qiskit_metal\analyses\quantization\lom_core_analysis.py: 170
 c:\Users\itsme\anaconda3\envs\qmetal\lib\site-packages\qiskit_metal\analyses\quantization\lom_core_analysis.py: 170
 c:\Users\itsme\anaconda3\envs\qmetal\lib\site-packages\qiskit_metal\analyses\quantization\lom_core_analysis.py: 170


Quantum object: dims = [[10, 3, 3, 3], [10, 3, 3, 3]], shape = (270, 270), type = oper, isherm = True
Qobj data =
[[-10975.96250154+0.00000000e+00j      0.        +9.72073787e-02j
       0.        +0.00000000e+00j ...      0.        +0.00000000e+00j
       0.        +0.00000000e+00j      0.        +0.00000000e+00j]
 [     0.        -9.72073787e-02j  -3975.96250154+0.00000000e+00j
       0.        +1.37471993e-01j ...      0.        +0.00000000e+00j
       0.        +0.00000000e+00j      0.        +0.00000000e+00j]
 [     0.        +0.00000000e+00j      0.        -1.37471993e-01j
    3024.03749846+0.00000000e+00j ...      0.        +0.00000000e+00j
       0.        +0.00000000e+00j      0.        +0.00000000e+00j]
 ...
 [     0.        +0.00000000e+00j      0.        +0.00000000e+00j
       0.        +0.00000000e+00j ...  57138.12174359+0.00000000e+00j
       0.        +4.68247310e+02j      0.        +0.00000000e+00j]
 [     0.        +0.00000000e+00j      0.        +0.00000000e+00j
   

In [26]:

hamiltonian_results = composite_sys.hamiltonian_results(hilbertspace, evals_count=30)


system frequencies in GHz:
--------------------------
{'transmon1_sys': 5.128366984427728, 'coup1_sys': 7.405254582888727, 'coup2_sys': 7.201399507829482, 'readout_sys': 7.004839150418283}

Chi matrices in MHz
--------------------------
               transmon1_sys  coup1_sys  coup2_sys  readout_sys
transmon1_sys    -305.484697  -1.438824  -1.023920    -2.448419
coup1_sys          -1.438824   2.091841   0.081559    -0.004331
coup2_sys          -1.023920   0.081559   2.027718     0.003093
readout_sys        -2.448419  -0.004331   0.003093     2.805109


In [ ]:
chi_df = hamiltonian_results['chi_in_MHz'].to_dataframe()
print("Transmon frequency     :", hamiltonian_results['fQ_in_Ghz']['transmon1_sys'], "GHz")
print("Transmon Anharmonicity :", chi_df['transmon1_sys']['transmon1_sys'], "MHz")
print("Readou chi             :", chi_df['readout_sys']['transmon1_sys'], "MHz")

In [27]:

composite_sys.compute_gs().to_dataframe()

,j1,coupler1,coupler2,readout
j1,0.000000,85.328990,83.275897,-97.207505
coupler1,85.328990,0.000000,17.074252,10.685905
coupler2,83.275897,17.074252,0.000000,10.408605
readout,-97.207505,10.685905,10.408605,0.000000


In [28]:

transmon.h_params

defaultdict(dict,
            {'j1': {'EJ': 13621.792733898432,
              'EC': 270.6303337815318,
              'Q_zpf': 3.204353268e-19,
              'default_charge_op': Operator(op=array([[-22,   0,   0, ...,   0,   0,   0],
                     [  0, -21,   0, ...,   0,   0,   0],
                     [  0,   0, -20, ...,   0,   0,   0],
                     ...,
                     [  0,   0,   0, ...,  20,   0,   0],
                     [  0,   0,   0, ...,   0,  21,   0],
                     [  0,   0,   0, ...,   0,   0,  22]]), add_hc=False)}})

In [29]:

gui.main_window.close()

08:08PM 10s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: QWindowsNativeFileDialogBase::selectNameFilter: Invalid parameter '*.metal.py' not found in 'All Files (*)'.

08:08PM 10s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  WARNING: QWindowsNativeFileDialogBase::selectNameFilter: Invalid parameter '*.metal.py' not found in 'All Files (*)'.



True